In [48]:
import sqlite3

def create_table():
    con_tab = sqlite3.connect("ticket_prices.db")
    cur_tab = con_tab.cursor()
    cur_tab.execute("CREATE TABLE IF NOT EXISTS tbl(city,price)")
    con_tab.commit()
    con_tab.close()

def fetch_prices(city_nm):
    con_fet = sqlite3.connect("ticket_prices.db")
    cur_fet = con_fet.cursor()
    price = cur_fet.execute(f"select price from tbl where city = '{city_nm}'").fetchone()
    print(f"tool called fetch prices db hit")
    return f"The prices for destination city {city_nm} is {price[0]}"


def update_prices(data):
    con_upd = sqlite3.connect("ticket_prices.db")
    cur_upd = con_upd.cursor()
    cur_upd.executemany("INSERT INTO tbl VALUES(?,?)", data)
    con_upd.commit()
    con_upd.close()

def delete_data():
    con_del = sqlite3.connect("ticket_prices.db")
    cur_del = con_del.cursor()
    cur_del.execute("DELETE from tbl")
    con_del.commit()
    con_del.close()

def fetchall_prices():
    con_fet = sqlite3.connect("ticket_prices.db")
    cur_fet = con_fet.cursor()
    return(cur_fet.execute(f"select * from tbl").fetchall())

In [27]:
data = [("Mumbai","$799"),("Hyderabad","$499")]
# update_prices(data)
con_upd = sqlite3.connect("ticket_prices.db")
cur_upd = con_upd.cursor()
cur_upd.executemany("INSERT INTO tbl VALUES(?,?)", [("Mumbai","$799"),("Hyderabad","$499")])
con_upd.commit()
con_upd.close()

In [44]:
# fetchall_prices()
# delete_data()
fetch_prices('Hyderabad')

'The prices for destination city Hyderabad is $499'

In [23]:
from openai import OpenAI
import json

ollma_url = "http://localhost:11434/v1"
ollma_key = "Ollama api secret key"

ollma = OpenAI(base_url=ollma_url, api_key=ollma_key)


In [17]:
# !ollama pull llama3.1:8b
!ollama list
# !ollama rm phi3:3.8b 

NAME               ID              SIZE      MODIFIED    
llama3.1:8b        46e0c10c039e    4.9 GB    3 days ago     
llama3.2:latest    a80c4f17acd5    2.0 GB    3 weeks ago    
gemma3:270m        e7d36fb2c3b3    291 MB    4 weeks ago    


## Adding tools scripts to chat function

In [49]:
fetch_prices_function = {
    "name" : "fetch_prices",
    "description" : "Funtion retrieves ticket prices for each city",
    "parameters" : {
        "type" : "object",
        "properties" : {
            "city_name":{
                "type" : "string",
                "description" : "Input takes the city customer wants to travel to"
            }
        },
        "required" : ["city_name"],
        "additionalparameters" : False
    }   
}

tool = [{"type":"function", "function":fetch_prices_function}]

In [50]:
system_prompt = """
You are an helpful assitant for flight booking
Will support users with flight ticket pricing details and also update latest price details into database
"""

def chat_func(message, history):
    messages = [{"role": "system","content":system_prompt}] + [{"role":h.get("role"), "content":h.get("content")} for h in history]
    messages += [{"role": "user","content": message}]

    resp = ollma.chat.completions.create(
        model='llama3.1:8b',
        messages=messages,
        tools=tool
    )

    if resp.choices[0].finish_reason == "tool_calls":
        for tool_call in resp.choices[0].message.tool_calls:
            if tool_call.function.name == 'fetch_prices':
                arguments = json.loads(tool_call.function.arguments)
                if arguments.get("city_name") is None:
                    prices = fetchall_prices()
                else:
                    prices = fetch_prices(arguments.get("city_name"))
                tool_resp = [{
                    "role": "tool",
                    "content": prices,
                    "tool_call_id": tool_call.id
                }]
                messages += tool_resp
        
        new_resp = ollma.chat.completions.create(model='llama3.1:8b',messages=messages)
        return f"Tool calls {new_resp.choices[0].message.content}"
    
    return resp.choices[0].message.content

In [ ]:
import gradio as gr 

system_prompt = """
You are an helpful assitant for flight booking
Will support users with flight ticket pricing details and also update latest price details into database
"""

gr.ChatInterface(
    fn=chat_func, type="messages"
).launch() 

* Running on local URL:  http://127.0.0.1:7867
* To create a public link, set `share=True` in `launch()`.


tool called fetch prices db hit
tool called fetch prices db hit
tool called fetch prices db hit
tool called fetch prices db hit
tool called fetch prices db hit
